In [2]:
ls

drive/  sample_data/


In [1]:
!pip uninstall torch

Found existing installation: torch 1.9.0+cu102
Uninstalling torch-1.9.0+cu102:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/lib/python3.7/dist-packages/caffe2/*
    /usr/local/lib/python3.7/dist-packages/torch-1.9.0+cu102.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torch/*
Proceed (y/n)? y
  Successfully uninstalled torch-1.9.0+cu102


In [3]:
!pip install torch==1.6.0
!pip install torchvision==0.7.0
!pip install coremltools

     |████████████████████████████████| 748.8 MB 15 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.6.0 which is incompatible.
     |████████████████████████████████| 5.9 MB 5.3 MB/s 
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
     |████████████████████████████████| 3.4 MB 5.4 MB/s 
  Created wheel for attr: filename=attr-0.3.1-py3-none-any.whl size=2457 sha256=1bd7f5b45b15d69a1cdc664eeeac2c02684bd570fcb0fb229584b262d95fb0b6
  Stored in directory: /root/.cache/pip/wheels/3b/5d/58/41fbe92f47031641008bd8559ee89e58bf0f123f9c18dea1cb
Successfully built attr


In [4]:
import torch
import torchvision

from torchvision import transforms
from PIL import Image
import numpy as np

import coremltools as ct

%matplotlib inline
from matplotlib.pyplot import imshow

model = torch.load('./drive/MyDrive/Flask/milk_epoch250_cpu.pt').cpu().eval()

input_image = Image.open('./drive/MyDrive/Flask/흰우유255.jpg')
# imshow(np.asarray(input_image))

preprocess = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225]
            ),
        ])

input_tensor = preprocess(input_image)
image_batch = input_tensor.unsqueeze(0)

with torch.no_grad():
    output = model(image_batch)
    _, preds = torch.max(output, 1)

traced_model = torch.jit.trace(model, image_batch).cpu()

class_labels = ["0", "1", "2", "3", "4"]

# Convert to Core ML using the Unified Conversion API
mlmodel = ct.convert(
    traced_model,
    inputs=[ct.ImageType(name="Image", shape=image_batch.shape)],
	classifier_config = ct.ClassifierConfig(class_labels)
)

# Save model
mlmodel.save("milk.mlmodel")

Translating MIL ==> MLModel Ops: 100%|██████████| 268/268 [00:01<00:00, 161.62 ops/s] 


In [5]:
mv milk.mlmodel drive/MyDrive/Flask/